In [2]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


def build_confusion_matrix(dataset, model):
        X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:, :-1], dataset.iloc[:, -1], test_size=0.25)
        predictions = model.fit(X_train, y_train).predict(X_test)
        tn, fn, fp, tp = confusion_matrix(predictions, y_test.values).ravel()
        return {'true_negative': tn, 'false_negative': fn, 'false_positive': fp, 'true_positive': tp}

def sensitivity(cm):
    return cm['true_positive'] / (cm['true_positive'] + cm['false_negative'])

def specificity(cm):
    return cm['true_negative'] / ( cm['true_negative'] + cm['false_positive'] )

def false_positive_rate(cm):
    return (1.0 - (specificity(cm)) )

def generate_scores(dataset, model):
    specificity_scores = []
    sensitivity_scores = []

    for n in range(1000):
        cm = build_confusion_matrix(dataset, model)
        specificity_scores.append(specificity(cm))
        sensitivity_scores.append(sensitivity(cm))
    
    spec_avg = pd.Series(specificity_scores).mean()
    sens_avg = pd.Series(sensitivity_scores).mean()
    
    return {'specificity': spec_avg, 'sensitivity': sens_avg }

def summarise_test(test):
    x ='For {}, in model {}\nAverage NO SIGFALL accuracy is {}\nAverage SIGFALL accuracy is {}\n---\n'
    print(x.format(test['data_name'],test['model'],test['specificity'],test['sensitivity']))

def balance_data_set(dataset, classifier):
    sigfall_indexes = dataset.index[dataset[classifier] == 0].to_list()
    n_falls = len(dataset) - len(sigfall_indexes)
    drop = len(sigfall_indexes) - n_falls
    random.shuffle(sigfall_indexes)
    drop_indexes = sigfall_indexes[0:drop]
    return dataset.drop(drop_indexes)

# Balance_data_generate_scores() is a seperate function to invoke a new randomly balanced dataset in each iteration.
# If a dataset was pre-balanced then run through generate_scores() it would overfit to that specific dataset 

def balance_data_generate_scores(dataset, classifier, model):
    specificity_scores = []
    sensitivity_scores = []

    for n in range(1000):
        b_dataset = balance_data_set(dataset, classifier)
        cm = build_confusion_matrix(b_dataset, model)
        specificity_scores.append(specificity(cm))
        sensitivity_scores.append(sensitivity(cm))
    
    spec_avg = pd.Series(specificity_scores).mean()
    sens_avg = pd.Series(sensitivity_scores).mean()
    
    return {'specificity': spec_avg, 'sensitivity': sens_avg }

In [6]:
# Load data sets
inc_updrs_sigfall = pd.read_csv('./working_data/normalised_increase_updrs_sigfall.csv')
inc_updrs_sigfall_raw = pd.read_csv('./working_data/increase_updrs_sigfall_raw.csv')
delta_updrs_sigfall = pd.read_csv('./working_data/normalised_delta_updrs_sigfall.csv')
delta_updrs_sigfall_raw = pd.read_csv('./working_data/delta_updrs_sigfall_raw.csv')
updrs_future_sigfall = pd.read_csv('./working_data/normalised_updrs_future_sigfall.csv')
updrs_future_sigfall_raw = pd.read_csv('./working_data/updrs_future_sigfall_raw.csv')

datasets = [{'data':inc_updrs_sigfall, 'name':'inc_updrs_sigfall'},
            {'data':inc_updrs_sigfall_raw, 'name':'inc_updrs_sigfall_raw'},
            {'data':delta_updrs_sigfall , 'name':'delta_updrs_sigfall'},
            {'data':delta_updrs_sigfall_raw , 'name':'delta_updrs_sigfall_raw'},
            {'data':updrs_future_sigfall , 'name':'updrs_future_sigfall'},
            {'data':updrs_future_sigfall_raw , 'name':'updrs_future_sigfall_raw'}
           ]

In [4]:
# Define models 
models = [
    LinearDiscriminantAnalysis(solver="svd"),
    LinearDiscriminantAnalysis(solver="lsqr"),
    LinearDiscriminantAnalysis(solver="eigen")
]


In [8]:
# Run tests

tests = []
for m in models:
    for d in datasets:
        tests.append({
            'data_name': d['name'],
            'dataset': d['data'],
            'model': m
        })

for t in tests:
    s = generate_scores(dataset=t['dataset'], model=t['model'])
    t['specificity'] = s['specificity']
    t['sensitivity'] = s['sensitivity']
    summarise_test(t)

For inc_updrs_sigfall, in model LinearDiscriminantAnalysis()
Average NO SIGFALL accuracy is 0.9776946025409408
Average SIGFALL accuracy is 0.12007940234976403
---

For inc_updrs_sigfall_raw, in model LinearDiscriminantAnalysis()
Average NO SIGFALL accuracy is 0.9789396584431258
Average SIGFALL accuracy is 0.11473078831966432
---

For delta_updrs_sigfall, in model LinearDiscriminantAnalysis()
Average NO SIGFALL accuracy is 0.9951955087105154
Average SIGFALL accuracy is 0.03445521863087324
---

For delta_updrs_sigfall_raw, in model LinearDiscriminantAnalysis()
Average NO SIGFALL accuracy is 0.9952371579606898
Average SIGFALL accuracy is 0.03439411465829157
---

For updrs_future_sigfall, in model LinearDiscriminantAnalysis()
Average NO SIGFALL accuracy is 0.9839340988425781
Average SIGFALL accuracy is 0.07352876867248571
---

For updrs_future_sigfall_raw, in model LinearDiscriminantAnalysis()
Average NO SIGFALL accuracy is 0.9835045153478515
Average SIGFALL accuracy is 0.07258726401658536

LinAlgError: The leading minor of order 14 of B is not positive definite. The factorization of B could not be completed and no eigenvalues or eigenvectors were computed.

In [7]:
# See QDA notebook for implementation of 'Balanced Datasets' - not sure its a valid thing to do ATM